<a href="https://colab.research.google.com/github/Vedmani/wear-particle-classification/blob/main/NasNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install wandb -qU
!wandb login

     |████████████████████████████████| 1.9 MB 13.3 MB/s 
     |████████████████████████████████| 166 kB 62.4 MB/s 
     |████████████████████████████████| 182 kB 60.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
     |████████████████████████████████| 166 kB 48.8 MB/s 
     |████████████████████████████████| 162 kB 73.8 MB/s 
     |████████████████████████████████| 162 kB 71.3 MB/s 
     |████████████████████████████████| 158 kB 68.6 MB/s 
     |████████████████████████████████| 157 kB 76.1 MB/s 
     |████████████████████████████████| 157 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 68.3 MB/s 
     |████████████████████████████████| 157 kB 72.1 MB/s 
     |████████████████████████████████| 157 kB 64.6 MB/s 
     |████████████████████████████████| 157 kB 74.6 MB/s 
     |████████████████████████████████| 156 kB 74.5 MB/s 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import wandb
from wandb.keras import WandbCallback

In [4]:
tf.keras.utils.set_random_seed(1)
tf.config.experimental.enable_op_determinism()

In [5]:
PATH = '/content/drive/MyDrive/Research Internship/final data/wear particles/Train'
test_path = '/content/drive/MyDrive/Research Internship/final data/wear particles/Test'
batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  PATH,
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
  test_path,
  seed = 42,
  image_size=(img_height, img_width))

class_names = train_ds.class_names
test_class_names = test_ds.class_names
num_classes = len(class_names)
print(class_names)

Found 879 files belonging to 5 classes.
Using 704 files for training.
Found 879 files belonging to 5 classes.
Using 175 files for validation.
Found 94 files belonging to 5 classes.
['cutting wear', 'normal rubbing and cutting wear combined', 'normal rubbing wear', 'red oxide', 'severe sliding wear']


In [6]:
nasnetMobile = tf.keras.applications.NASNetMobile(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax",
)
model_nasnetMobile=Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  nasnetMobile,
  #layers.Flatten(),
  #layers.Dense(32, activation='relu'),
  layers.Dropout(0.2), 
  layers.Dense(num_classes, activation='softmax')
])
nasnetMobile.trainable=False
model_nasnetMobile.summary()

19993432/19993432 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 NASNet (Functional)         (None, 1056)              4269716   
                                                                 
 dropout (Dropout)           (None, 1056)              0         
                                                                 
 dense (Dense)               (None, 5)                 5285      
                                                                 
Total params: 4,275,001
Trainable params: 5,285
Non-trainable params: 4,269,716
_________________________________________________________________


In [7]:
model_nasnetMobile.compile(
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam'),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy']
              )

In [12]:
wandb.init(
      # Set the project where this run will be logged
      project="Transfer_Learning_New", 
      name = "NASNetMobile",
      )

In [13]:
%%time
epochs=40
history = model_nasnetMobile.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
callbacks=[WandbCallback()]
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/40
22/22 [==============================] - ETA: 0s - loss: 1.1591 - accuracy: 0.5511

wandb: Adding directory to artifact (/content/wandb/run-20221106_082332-1lpsvw5n/files/model-best)... Done. 0.2s


22/22 [==============================] - 333s 11s/step - loss: 1.1591 - accuracy: 0.5511 - val_loss: 0.8263 - val_accuracy: 0.7029
Epoch 2/40
22/22 [==============================] - ETA: 0s - loss: 0.6111 - accuracy: 0.8239

wandb: Adding directory to artifact (/content/wandb/run-20221106_082332-1lpsvw5n/files/model-best)... Done. 0.2s


...

22/22 [==============================] - 89s 4s/step - loss: 0.0280 - accuracy: 0.9943 - val_loss: 0.0742 - val_accuracy: 0.9714
Epoch 39/40
22/22 [==============================] - ETA: 0s - loss: 0.0263 - accuracy: 0.9972

wandb: Adding directory to artifact (/content/wandb/run-20221106_082332-1lpsvw5n/files/model-best)... Done. 0.2s


22/22 [==============================] - 89s 4s/step - loss: 0.0263 - accuracy: 0.9972 - val_loss: 0.0738 - val_accuracy: 0.9714
Epoch 40/40
22/22 [==============================] - ETA: 0s - loss: 0.0252 - accuracy: 1.0000

wandb: Adding directory to artifact (/content/wandb/run-20221106_082332-1lpsvw5n/files/model-best)... Done. 0.2s


22/22 [==============================] - 89s 4s/step - loss: 0.0252 - accuracy: 1.0000 - val_loss: 0.0722 - val_accuracy: 0.9829
CPU times: user 57min 39s, sys: 1min 28s, total: 59min 7s
Wall time: 1h 18min 7s


In [14]:
wandb.finish()

accuracy,▁▅▆▇▇▇██████████████████████████████████
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▅▆▇▇▇▇██▇██████████████████████████████
val_loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,1.0
best_epoch,39
best_val_loss,0.07217
epoch,39
loss,0.0252
val_accuracy,0.98286


In [15]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
Tesla T4
